In [1]:
import pandas as pd
import numpy as np
from collections import Counter

# Preparations

In [2]:
submissions = pd.read_csv('test_task_data.csv', sep=";")
submissions

,907,1909,1929,1790.87,00:50:00,20:00:00
0,908,1909,1929,1790.87,00:50:00,20:00:00
1,909,1929,1909,1790.87,10:01:00,01:00:00
2,909,1929,1921,199.15,11:10:00,01:00:00
3,910,1929,1909,1790.87,10:01:00,01:00:00
4,910,1929,1921,199.15,11:10:00,01:00:00
...,...,...,...,...,...,...
244,8501,1902,1937,195.00,05:50:00,21:50:00
245,8701,1902,1981,193.55,05:55:00,21:10:00
246,8801,1902,1981,193.40,06:35:00,19:30:00
247,8901,1937,1902,236.00,21:35:00,15:25:00


In [3]:
submissions.columns = ["train_number", "station_depature", "station_arrive", "coast", "time_depature", "time_arrive"]

In [4]:
submissions.head(5)

,train_number,station_depature,station_arrive,coast,time_depature,time_arrive
0,908,1909,1929,1790.87,00:50:00,20:00:00
1,909,1929,1909,1790.87,10:01:00,01:00:00
2,909,1929,1921,199.15,11:10:00,01:00:00
3,910,1929,1909,1790.87,10:01:00,01:00:00
4,910,1929,1921,199.15,11:10:00,01:00:00


In [5]:
# add last row
df2 = {"train_number": 907, "station_depature":1909, "station_arrive":1929, "coast":1790.87,  "time_depature":"00:50:00", "time_arrive":"20:00:00"}
df = submissions.append(df2, ignore_index=True)
df

,train_number,station_depature,station_arrive,coast,time_depature,time_arrive
0,908,1909,1929,1790.87,00:50:00,20:00:00
1,909,1929,1909,1790.87,10:01:00,01:00:00
2,909,1929,1921,199.15,11:10:00,01:00:00
3,910,1929,1909,1790.87,10:01:00,01:00:00
4,910,1929,1921,199.15,11:10:00,01:00:00
...,...,...,...,...,...,...
245,8701,1902,1981,193.55,05:55:00,21:10:00
246,8801,1902,1981,193.40,06:35:00,19:30:00
247,8901,1937,1902,236.00,21:35:00,15:25:00
248,9101,1902,1921,160.56,05:15:00,20:50:00}


In [6]:
len(df["station_depature"].unique()), len(df["station_arrive"].unique())

(6, 6)

In [7]:
df["station_depature"].unique(), df["station_arrive"].unique()

(array([1909, 1929, 1902, 1937, 1981, 1921], dtype=int64),
 array([1929, 1909, 1921, 1937, 1902, 1981], dtype=int64))

# Coast

In [8]:
arr = []
totalSum = 0
stationsQuantity = len(df["station_depature"].unique())
dfSized = pd.DataFrame(columns=df.drop(["time_depature", "time_arrive"], axis=1).columns)

In [9]:
dfSized = df.groupby(["station_depature", "station_arrive"],as_index=False).agg({"coast":["min"]})

In [10]:
dfSized.reset_index().drop(["index"], axis=1)

c:\program files\python39\lib\site-packages\pandas\core\generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


,station_depature,station_arrive,coast
,,,min
0,1902,1921,114.11
1,1902,1929,164.65
2,1902,1937,131.34
3,1902,1981,130.60
4,1909,1921,1407.37
5,1909,1929,1790.87
6,1921,1902,114.12
7,1921,1909,1421.12
8,1921,1929,130.36


In [11]:
globalArr = []

In [12]:
# for each row (stationsQuantity - 2) times
def foo(currentStation, totalSum, counter, arr):
    print("currentStation ", currentStation, "totalSum", totalSum)
    if counter == stationsQuantity-1:
        print("final step")
        arr.append(round(totalSum, 2))
        globalArr.append(arr)
        return 
    for index, value in enumerate(dfSized[dfSized["station_depature"]==currentStation]["station_arrive"]):
        value = int(value)
        if value not in arr:
            Sum, *y = dfSized[(dfSized["station_depature"]==currentStation) & (dfSized["station_arrive"]==value)]["coast"]["min"]
            totalSum += Sum
            arr2 = arr.copy()
            arr2.append(value)
            counter += 1
            foo(value, totalSum, counter, arr2)
            counter -= 1
            totalSum -= Sum
    return

In [13]:
for index, value in enumerate(dfSized["station_depature"]):
    # func for each row
    totalSum = 0
    print("Main loop ", value)
    arrStations = [int(value)]
    foo(value, totalSum, 0, arrStations)

Main loop  1902
currentStation  1902 totalSum 0
currentStation  1921 totalSum 114.11
currentStation  1909 totalSum 1535.2299999999998
currentStation  1929 totalSum 3326.0999999999995
currentStation  1937 totalSum 3460.2099999999996
currentStation  1981 totalSum 3600.7199999999993
final step
currentStation  1981 totalSum 3638.5199999999995
currentStation  1937 totalSum 3779.4999999999995
final step
currentStation  1929 totalSum 244.4699999999999
currentStation  1909 totalSum 2035.3399999999997
currentStation  1937 totalSum 378.5799999999998
currentStation  1981 totalSum 519.0899999999998
currentStation  1981 totalSum 556.8899999999999
currentStation  1937 totalSum 697.8699999999999
currentStation  1937 totalSum 254.6199999999999
currentStation  1929 totalSum 388.7299999999999
currentStation  1909 totalSum 2179.6
currentStation  1981 totalSum 701.1500000000001
currentStation  1981 totalSum 395.1299999999999
currentStation  1929 totalSum 679.9699999999998
currentStation  1909 totalSum 247

In [14]:
globalArr

[[1902, 1921, 1909, 1929, 1937, 1981, 3600.72],
 [1902, 1921, 1909, 1929, 1981, 1937, 3779.5],
 [1902, 1921, 1937, 1981, 1929, 1909, 2470.84],
 [1902, 1921, 1981, 1937, 1929, 1909, 2372.1],
 [1902, 1929, 1909, 1921, 1937, 1981, 3643.91],
 [1902, 1929, 1909, 1921, 1981, 1937, 3695.9],
 [1902, 1929, 1937, 1981, 1921, 1909, 2174.74],
 [1902, 1929, 1981, 1937, 1921, 1909, 2273.08],
 [1902, 1937, 1921, 1909, 1929, 1981, 3889.66],
 [1902, 1937, 1921, 1981, 1929, 1909, 2632.99],
 [1902, 1937, 1929, 1909, 1921, 1981, 3655.72],
 [1902, 1937, 1929, 1981, 1921, 1909, 2313.34],
 [1902, 1937, 1981, 1921, 1909, 1929, 3798.19],
 [1902, 1937, 1981, 1921, 1929, 1909, 2507.43],
 [1902, 1937, 1981, 1929, 1909, 1921, 3754.93],
 [1902, 1937, 1981, 1929, 1921, 1909, 2139.94],
 [1902, 1981, 1921, 1909, 1929, 1937, 3791.05],
 [1902, 1981, 1921, 1937, 1929, 1909, 2510.44],
 [1902, 1981, 1929, 1909, 1921, 1937, 3754.19],
 [1902, 1981, 1929, 1937, 1921, 1909, 2204.58],
 [1902, 1981, 1937, 1921, 1909, 1929, 3717.

In [15]:
bestResultsCoast = []
globalArrNp = np.array(globalArr.copy())

In [16]:
globalArrNp

array([[1902.  , 1921.  , 1909.  , ..., 1937.  , 1981.  , 3600.72],
       [1902.  , 1921.  , 1909.  , ..., 1981.  , 1937.  , 3779.5 ],
       [1902.  , 1921.  , 1937.  , ..., 1929.  , 1909.  , 2470.84],
       ...,
       [1981.  , 1937.  , 1921.  , ..., 1929.  , 1902.  , 3751.55],
       [1981.  , 1937.  , 1929.  , ..., 1921.  , 1909.  , 1974.99],
       [1981.  , 1937.  , 1929.  , ..., 1921.  , 1902.  , 3587.45]])

In [17]:
sumColumn = globalArrNp[:, -1]
minSum = sumColumn.min()
minSum

1927.18

In [18]:
for array in globalArr:
    if array[-1] == minSum and array not in bestResultsCoast:
        bestResultsCoast.append(array)

## Coast — Result

In [19]:
print(f"Path {bestResultsCoast[0][:-1]} with cheapest coast {bestResultsCoast[0][-1]}")

Path [1909, 1921, 1902, 1981, 1937, 1929] with cheapest coast 1927.18


# Timestamp

In [20]:
df.head(5)

,train_number,station_depature,station_arrive,coast,time_depature,time_arrive
0,908,1909,1929,1790.87,00:50:00,20:00:00
1,909,1929,1909,1790.87,10:01:00,01:00:00
2,909,1929,1921,199.15,11:10:00,01:00:00
3,910,1929,1909,1790.87,10:01:00,01:00:00
4,910,1929,1921,199.15,11:10:00,01:00:00


In [21]:
df.iloc[1, :]

train_number             909
station_depature        1929
station_arrive          1909
coast                1790.87
time_depature       10:01:00
time_arrive         01:00:00
Name: 1, dtype: object

In [22]:
dep, arrive = pd.to_datetime(df.iloc[1, :]["time_depature"]), pd.to_datetime(df.iloc[1, :]["time_arrive"])
res = dep - arrive
res.value

32460000000000

In [23]:
df.at[248, 'time_arrive'] = "20:50:00"

In [24]:
df.iloc[248, :]

train_number            9101
station_depature        1902
station_arrive          1921
coast                 160.56
time_depature       05:15:00
time_arrive         20:50:00
Name: 248, dtype: object

In [25]:
df["duration_trip"] = df.apply(lambda row: int(abs(pd.to_datetime(row["time_arrive"]) - pd.to_datetime(row["time_depature"])).value), axis=1)

In [26]:
df.head(5)

,train_number,station_depature,station_arrive,coast,time_depature,time_arrive,duration_trip
0,908,1909,1929,1790.87,00:50:00,20:00:00,69000000000000
1,909,1929,1909,1790.87,10:01:00,01:00:00,32460000000000
2,909,1929,1921,199.15,11:10:00,01:00:00,36600000000000
3,910,1929,1909,1790.87,10:01:00,01:00:00,32460000000000
4,910,1929,1921,199.15,11:10:00,01:00:00,36600000000000


In [27]:
dfSizedTime = df.groupby(["station_depature", "station_arrive"],as_index=False).agg({"duration_trip":["min"]})

In [28]:
dfSizedTime.head(5)

,station_depature,station_arrive,duration_trip
,,,min
0,1902,1921,16440000000000
1,1902,1929,16380000000000
2,1902,1937,21900000000000
3,1902,1981,17640000000000
4,1909,1921,12600000000000


In [29]:
f,*y = dfSizedTime[(dfSized["station_depature"]==1902) & (dfSized["station_arrive"]==1929)]["duration_trip"]["min"]
f

16380000000000

In [30]:
globalArrTime = []
totalTime = 0

In [31]:
# for each row (stationsQuantity - 2) times
def foo(currentStation, totalTime, counter, arr):
    print("currentStation ", currentStation, "totalSum", totalTime)
    if counter == stationsQuantity-1:
        print("final step")
        arr.append(round(totalTime, 2))
        globalArrTime.append(arr)
        return 
    for index, value in enumerate(dfSized[dfSized["station_depature"]==currentStation]["station_arrive"]):
        value = int(value)
        if value not in arr:
            Time, *y = dfSizedTime[(dfSizedTime["station_depature"]==currentStation) & (dfSizedTime["station_arrive"]==value)]["duration_trip"]["min"]
            totalTime += Time
            arr2 = arr.copy()
            arr2.append(value)
            counter += 1
            foo(value, totalTime, counter, arr2)
            counter -= 1
            totalTime -= Time
    return

In [32]:
for index, value in enumerate(dfSized["station_depature"]):
    # func for each row
    totalTime = 0
    print("Main loop ", value)
    arrStations = [int(value)]
    foo(value, totalSum, 0, arrStations)

Main loop  1902
currentStation  1902 totalSum 0
currentStation  1921 totalSum 16440000000000
currentStation  1909 totalSum 16920000000000
currentStation  1929 totalSum 64320000000000
currentStation  1937 totalSum 100620000000000
currentStation  1981 totalSum 144780000000000
final step
currentStation  1981 totalSum 67980000000000
currentStation  1937 totalSum 111060000000000
final step
currentStation  1929 totalSum 41160000000000
currentStation  1909 totalSum 73620000000000
currentStation  1937 totalSum 77460000000000
currentStation  1981 totalSum 121620000000000
currentStation  1981 totalSum 44820000000000
currentStation  1937 totalSum 87900000000000
currentStation  1937 totalSum 39300000000000
currentStation  1929 totalSum 82920000000000
currentStation  1909 totalSum 115380000000000
currentStation  1981 totalSum 86580000000000
currentStation  1981 totalSum 83460000000000
currentStation  1929 totalSum 85740000000000
currentStation  1909 totalSum 118200000000000
final step
currentStatio

In [33]:
globalArrTime

[[1902, 1921, 1909, 1929, 1937, 1981, 144780000000000],
 [1902, 1921, 1909, 1929, 1981, 1937, 111060000000000],
 [1902, 1921, 1937, 1981, 1929, 1909, 118200000000000],
 [1902, 1921, 1981, 1937, 1929, 1909, 155160000000000],
 [1902, 1929, 1909, 1921, 1937, 1981, 128460000000000],
 [1902, 1929, 1909, 1921, 1981, 1937, 124080000000000],
 [1902, 1929, 1937, 1981, 1921, 1909, 119880000000000],
 [1902, 1929, 1981, 1937, 1921, 1909, 88500000000000],
 [1902, 1937, 1921, 1909, 1929, 1981, 98340000000000],
 [1902, 1937, 1921, 1981, 1929, 1909, 101100000000000],
 [1902, 1937, 1929, 1909, 1921, 1981, 130140000000000],
 [1902, 1937, 1929, 1981, 1921, 1909, 92220000000000],
 [1902, 1937, 1981, 1921, 1909, 1929, 136500000000000],
 [1902, 1937, 1981, 1921, 1929, 1909, 145800000000000],
 [1902, 1937, 1981, 1929, 1909, 1921, 113400000000000],
 [1902, 1937, 1981, 1929, 1921, 1909, 93720000000000],
 [1902, 1981, 1921, 1909, 1929, 1937, 124380000000000],
 [1902, 1981, 1921, 1937, 1929, 1909, 13914000000000

In [34]:
bestResultsTime = []
globalArrTimeNp = np.array(globalArrTime.copy())

In [35]:
sumTimeColumn = globalArrTimeNp[:, -1]
minSum = sumTimeColumn.min()
minSum

64080000000000

In [36]:
for array in globalArrTime:
    if array[-1] == minSum and array not in bestResultsTime:
        bestResultsTime.append(array)

In [37]:
import datetime

In [38]:
bestTime = str(datetime.timedelta(seconds=(bestResultsTime[0][-1]/10**9)))

In [39]:
print(f"Best path {bestResultsTime[0][:-1]} with time {bestTime}")

Best path [1937, 1902, 1929, 1981, 1921, 1909] with time 17:48:00
